In [1]:
#Question 1
import gzip
from collections import defaultdict

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)
    
businessCount = defaultdict(int)
userinformation = defaultdict(list)
userinformation_train = defaultdict(list)
iteminformation_train = defaultdict(list)
userinformation_train_raw = defaultdict(list)
iteminformation_train_raw = defaultdict(list)
userinformation_validation = defaultdict(list)
userrating_train = []
iteminformation =defaultdict(list)
UR =defaultdict(list)
userrating_validation = []
userrating_validation_check = defaultdict(list)
allRatings = []
userRatings = defaultdict(list)
train_set =set()
validation_set =set()
count = 0
all_set = set()
for l in readGz("train.json.gz"):
    user,item = l['reviewerID'],l['itemID']
    userinformation[user].append(item)
    iteminformation[item].append(user)
    allRatings.append(l['rating'])
    all_set.add(l['reviewerID'] + '-' + l['itemID'])
    if count < 100000:
        user,business = l['reviewerID'],l['itemID']
        userinformation_train[user].append(l['rating'])
        iteminformation_train[item].append(l['rating'])
        train_set.add(l['reviewerID'] + '-' + l['itemID'] )
        userinformation_train_raw[user].append(l['rating'])
        iteminformation_train_raw[item].append(l['rating'])
        userrating_train.append(l['rating'])
        count = count +1 
    else:
        user,business = l['reviewerID'],l['itemID']
        userinformation_validation[user].append(l['rating'])
        userrating_validation.append(l['rating'])
        validation_set.add(l['reviewerID'] + '-' + l['itemID'])
        userrating_validation_check[user].append(l['rating'])
        count = count +1 
        


In [2]:
avgrating_train = sum(userrating_train)/len(userrating_train)
avgrating_all = sum(allRatings)/len(allRatings)
print(avgrating_all)
4.23471

#so the avgrating_train ,what is alpha is 4.232

4.23471


4.23471

In [3]:
s= 0
for i in range(0,len(userrating_validation)):
    s +=(userrating_validation[i]-avgrating_train)**2
MSE = s/len(userrating_validation)
MSE
#MSE is 1.22248 if using average to predict validation set

1.222481119999121

In [4]:
#question 6
TR_dict = defaultdict(float)
UR_dict = defaultdict(list)
IR_dict = defaultdict(list)
count = 0
dict1 =defaultdict(list)
dict2 =defaultdict(list)

for l in all_set:
    TR_dict[l] = allRatings[count]
    count += 1
for i in TR_dict:
    UR_dict[i.split('-')[0]].append(TR_dict[i])
    IR_dict[i.split('-')[1]].append(TR_dict[i])
for USER in UR_dict:
    dict1[USER] = sum(UR_dict[USER])/len(UR_dict[USER])
for ITEM in IR_dict:
    dict2[ITEM] = sum(IR_dict[ITEM])/len(IR_dict[ITEM])

In [5]:
alpha = avgrating_all
beta_u = defaultdict(float)
beta_i = defaultdict(float)
for item in IR_dict:
    beta_i[item] = dict2[item] -alpha
for user in UR_dict:
    beta_u[user] = dict1[user] -alpha    

In [6]:
UI_dict = defaultdict(list)
IU_dict = defaultdict(list)

for l in train_set:
    UI_dict[l.split('-')[0]].append(l.split('-')[1])
    IU_dict[l.split('-')[1]].append(l.split('-')[0])

In [7]:
def converge(value, value_last, epsilon):
    error = 0
    for i in value:
        error += abs(value[i] - value_last[i])
    #print error
    if error < epsilon:
        return True
    else:
        return False


In [8]:
train_data = []
validation_data = []
index = 0
for l in readGz("train.json.gz"):
    if index < 100000:
        train_data.append(l)
    else:
        train_data.append(l)
        validation_data.append(l)
    index += 1

In [9]:
user_sample = []
item_sample = []
for l in readGz("train.json.gz"):
    user_sample.append(l['reviewerID'])
    item_sample.append(l['itemID'])




In [10]:
cat_sample = []
cat_rate =[]
for l in readGz("train.json.gz"):
    cat_sample.append(l['categories'])
    cat_rate.append(l['rating'])
IC_dict = dict(zip(item_sample,cat_sample))
UI_dict = userinformation
dict7 ={}
list3 = set()

In [11]:
for i in range(len(cat_sample)):
    for j in range(len(cat_sample[i])):
        for k in range(len(cat_sample[i][j])):
            list3.add(cat_sample[i][j][k])
    dict7[i] = list3
    list3 = set()
dict6 = {}
list2 =set()



In [12]:
for item in IC_dict:
    for i in range(0,len(IC_dict[item])):
        for k in range (0,len(IC_dict[item][i])):
            if len(IC_dict[item][i][k]) > 0:
                dict6[IC_dict[item][i][k]] = 0
for item in IC_dict:
    for i in range(0,len(IC_dict[item])):
        for k in range (0,len(IC_dict[item][i])):
            if len(IC_dict[item][i][k]) > 0:
                dict6[IC_dict[item][i][k]] += 1      

CR_dict= []
dict4 ={}
dict5={}

In [13]:
for k in dict6:
    dict4[k]= 0
    dict5[k]=0
for i in range(200000):
    CR_dict.append(cat_rate[i])
    CR_dict.append(dict7[i])
for i in range(200000):
    k = 2*i
    m = k+1
    for j in range(len(CR_dict[m])):
        list_object = list(CR_dict[m])
        dict5[list_object[j]] +=1
        dict4[list_object[j]] +=CR_dict[k]
for k in dict4:
    dict4[k] = dict4[k]/dict5[k]

In [14]:
beta_c = defaultdict(float)
alpha = avgrating_train
for cat in dict4:
    beta_c[cat]=dict4[cat] -alpha

In [16]:
    alpha_new = 0
    beta_u = defaultdict(float)
    beta_i = defaultdict(float)
    for item in IR_dict:
        beta_i[item] = dict2[item] -alpha
    for user in UR_dict:
        beta_u[user] = dict1[user] -alpha   
    beta_u_new = beta_u.copy()
    beta_i_new = beta_i.copy()
    lamda = 5
    count = 0
    epsilon =0.0001
    finish = False
    s=0
    cat_count1 =0
    cat_count2=0
    cat_count3 =0
    while count ==0 :
        num_train = len(train_data)
        for l in train_data:
            user,item = l['reviewerID'],l['itemID']
            cat= l['categories']
            for g in range(len(cat)):
                for b in range(len(cat[g])):
                    cat_count3 += 1
                    cat_count2 += beta_c[cat[g][b]]
            cat_count1 = cat_count2/cat_count3
            cat_count2 =0
            cat_count3 =0
            alpha_new += (l["rating"] - beta_u[user] - beta_i[item]-cat_count1)
        alpha_new /= num_train
        num_item = defaultdict(int)
        for user in beta_u_new:
            beta_u_new[user] = 0
        for l in train_data:
            user,item = l['reviewerID'],l['itemID']
            cat= l['categories']
            for g in range(len(cat)):
                for b in range(len(cat[g])):
                    cat_count3 += 1
                    cat_count2 += beta_c[cat[g][b]]
            cat_count1 = cat_count2/cat_count3
            cat_count2 =0
            cat_count3 =0
            num_item[user] += 1
            beta_u_new[user] += (l["rating"] - alpha_new - beta_i[item]-cat_count1)
        for user in beta_u:
            beta_u_new[user] /= (lamda + num_item[user])
        num_user = defaultdict(int)
        for item in beta_i_new:
            beta_i_new[item] = 0
        for l in train_data:
            user,item = l['reviewerID'],l['itemID']
            cat= l['categories']
            for g in range(len(cat)):
                for b in range(len(cat[g])):
                    cat_count3 += 1
                    cat_count2 += beta_c[cat[g][b]]
            cat_count1 = cat_count2/cat_count3
            cat_count2 =0
            cat_count3 =0
            num_user[item] += 1
            beta_i_new[item] += (l["rating"] - alpha_new - beta_u_new[user]-cat_count1)
        for item in beta_i_new:
            beta_i_new[item] /= (lamda + num_user[item])
        for l in train_data:
            user,item = l['reviewerID'],l['itemID']
            cat= l['categories']
            for g in range(len(cat)):
                for b in range(len(cat[g])):
                    cat_count3 += 1
                    cat_count2 += beta_c[cat[g][b]]
            cat_count1 = cat_count2/cat_count3
            cat_count2 =0
            cat_count3 =0
            num_user[item] += 1
            beta_i_new[item] += (l["rating"] - alpha_new - beta_u_new[user]-cat_count1)
        for item in beta_i_new:
            beta_i_new[item] /= (lamda + num_user[item])
        if abs(alpha - alpha_new) < epsilon and \
        converge(beta_u, beta_u_new, epsilon)\
        and converge(beta_i, beta_i_new, epsilon):
            finish = True
        beta_u = beta_u_new.copy()
        beta_i = beta_i_new.copy()
        prediction = []
        alpha = alpha_new
        for l in validation_data:
            user,item,rating,cat = l['reviewerID'],l['itemID'],l['rating'],l['categories']
            cat= l['categories']
            for g in range(len(cat)):
                for b in range(len(cat[g])):
                    cat_count3 += 1
                    cat_count2 += beta_c[cat[g][b]]
            cat_count1 = cat_count2/cat_count3
            if user in beta_u_new:
                user_judgement = True
            else:
                user_judgement = False
            if item in beta_i_new:
                item_judgement = True
            else:
                item_judgement = False
            if item_judgement == False and user_judgement == False :
                predict_value = alpha_new
            elif item_judgement == False and user_judgement == True:
                predict_value = alpha_new + beta_u_new[user]
            elif item_judgement == True and user_judgement ==False:
                predict_value = alpha_new + beta_i_new[item]+cat_count1
            else:
                predict_value = alpha_new + beta_i_new[item]+beta_u_new[user]+cat_count1
            s += (predict_value-rating)**2
            predict_value = 0
        MSE_new = s/len(validation_data)
        print(MSE_new)
        s = 0
        if finish == True:
            count = 1
        else:
            count = 0

0.8351118122379678
0.8308662302901886
0.8309694785353561
0.830981780606556
0.8309831658601913
0.8309833361864422
0.8309833632059401
0.8309833719678025
0.8309833771132137
0.8309833804715305
0.8309833825849395
0.8309833838581233
0.8309833846006037
0.8309833850238583
0.8309833852613395
0.8309833853930761
0.8309833854655403
0.8309833855051391
0.8309833855266763
0.8309833855383671
0.8309833855446906
0.8309833855480798


In [99]:
# question MSE_old is 1.5667
print ('Min MSE is ',MSE_old)
count= count -1
print(' iteration time is ',count)

Min MSE is  3
 iteration time is  0


In [17]:
test = open("pairs_Rating.txt","r").readlines()
test = test[1:]
test = [i.strip() for i in test]
prediction = []
point = 0

for i in test:
    user,item = i.strip().split('-')
    cat= l['categories']
    for g in range(len(cat)):
        for b in range(len(cat[g])):
            cat_count3 += 1
            cat_count2 += beta_c[cat[g][b]]
    cat_count1 = cat_count2/cat_count3
    if user in beta_u_new:
        user_judgement = True
    else:
        user_judgement = False
    if item in beta_i_new:
        item_judgement = True
    else:
        item_judgement = False
    if item_judgement == False and user_judgement == False :
        predict_value = alpha_new
    elif item_judgement == False and user_judgement == True:
        predict_value = alpha_new + beta_u_new[user]
    elif item_judgement == True and user_judgement ==False:
        predict_value = alpha_new + beta_i_new[item]+cat_count1
    else:
        predict_value = alpha_new + beta_i_new[item]+beta_u_new[user]+cat_count1
    prediction.append(predict_value)
    predict_value = 0
import csv

with open('prediction_rate.csv', 'w') as csv_file:
    csv_write = csv.writer(csv_file)
    for a in prediction:
        csv_write.writerow(str(a))

In [23]:
#question 7

for c in range(0,1000):
    beta_u_new = beta_u.copy()
    beta_i_new = beta_i.copy()
    lamda = 1*c
    alpha = 4.232
    beta_u_new = beta_u.copy()
    beta_i_new = beta_i.copy()
    alpha = 4.23471
    l = 0
    m = 0
    n = 0
while count > 0 :
    for number in TR_dict:
        l += TR_dict[number]
    for user in beta_u:
        m = beta_u_new[user] + m
    for item in beta_i:
        n += beta_i_new[item]
    alpha_new = (l-m-n)/200000
    l = 0
    m = 0
    n = 0
    for user in beta_u_new:
        count_item = []
        l = sum(UR_dict[user])
        m = alpha_new*len(UR_dict[user])
        for item in UI_dict[user]:
            count_item.append(item)
        for x in count_item:
            n += beta_i_new[x]
        beta_u_new[user] = (l - m -n)/(lamda + len(UI_dict[user]))
        l = 0
        m = 0
        n = 0
    for k in beta_i_new:
        count_user = []
        l = sum(IR_dict[k])
        m = alpha_new*len(IR_dict[k])
        for user in IU_dict[k]:
            count_user.append(user)
        for x in count_user:
            n += beta_u_new[x]
        beta_i_new[k] = (l - m -n)/(lamda + len(IU_dict[k]))
        l = 0
        m = 0
        n = 0  
    prediction = []
    for i in range(0,len(validation_user)):
        if validation_user[i] in beta_u_new:
            user_judgement = True
        else:
            user_judgement = False
        if validation_item[i] in beta_i_new:
            item_judgement = True
        else:
            item_judgement = False
        if item_judgement == False and user_judgement == False :
            predict_value = alpha_new
        elif item_judgement == False and user_judgement == True:
            predict_value = alpha_new + beta_u_new[validation_user[i]]
        elif item_judgement == True and user_judgement ==False:
            predict_value = alpha_new + beta_i_new[validation_item[i]]
        else:
            predict_value = alpha_new + beta_i_new[validation_item[i]]+beta_u_new[validation_user[i]]
        prediction.append(predict_value)
        predict_value = 0
    s= 0
    for i in range(0,len(prediction)):
        s +=(prediction[i]-userrating_validation[i])**2
    MSE_new = s/len(prediction)
    count = count -1
    print(MSE_new)

KeyboardInterrupt: 

In [10]:
max_user = max(beta_u_new,key=beta_u_new.get)
min_user = min(beta_u_new,key=beta_u_new.get)
max_item = max(beta_i_new,key=beta_i_new.get)
min_item = min(beta_i_new,key=beta_i_new.get)
print('The user has largest beta is ',max_user)
print('The user has smallest beta is ',min_user)
print('The item has largest beta is ',max_item)
print('The item has smallest beta is ',min_item)

The user has largest beta is  U606968682
The user has smallest beta is  U916648954
The item has largest beta is  I562830918
The item has smallest beta is  I284886896


In [ ]:
validation_user = []
validation_item = []
for d in validation_set:
        validation_user.append(d.split('-')[0])
        validation_item.append(d.split('-')[1])
MSE_list ={}
#Question 8
for c in range(0,1000):
    beta_u_new = beta_u.copy()
    beta_i_new = beta_i.copy()
    lamda = 1*c
    alpha = 4.232
    l = 0
    m = 0
    n = 0
    MSE_old = 3
    MSE_new= 2
    while MSE_new < MSE_old:
        MSE_old = MSE_new
        for number in TR_dict:
            l += TR_dict[number]
        for user in beta_u:
            m = beta_u_new[user] + m
        for item in beta_i:
            n += beta_i_new[item]
        alpha_new = (l-m-n)/100000
        l = 0
        m = 0
        n = 0
        for user in beta_u_new:
            count_item = []
            l = sum(UR_dict[user])
            m = alpha_new*len(UR_dict[user])
            for item in UI_dict[user]:
                count_item.append(item)
            for x in count_item:
                n += beta_i_new[x]
            beta_u_new[user] = (l - m -n)/(lamda + len(UI_dict[user]))
            l = 0
            m = 0
            n = 0
        for k in beta_i_new:
            count_user = []
            l = sum(IR_dict[k])
            m = alpha_new*len(IR_dict[k])
            for user in IU_dict[k]:
                count_user.append(user)
            for x in count_user:
                n += beta_u_new[x]
            beta_i_new[k] = (l - m -n)/(lamda + len(IU_dict[k]))
            l = 0
            m = 0
            n = 0  
        prediction = []
        for i in range(0,len(validation_user)):
            if validation_user[i] in beta_u_new:
                user_judgement = True
            else:
                user_judgement = False
            if validation_item[i] in beta_i_new:
                item_judgement = True
            else:
                item_judgement = False
            if item_judgement == False and user_judgement == False :
                predict_value = alpha_new
            elif item_judgement == False and user_judgement == True:
                predict_value = alpha_new + beta_u_new[validation_user[i]]
            elif item_judgement == True and user_judgement ==False:
                predict_value = alpha_new + beta_i_new[validation_item[i]]
            else:
                predict_value = alpha_new + beta_i_new[validation_item[i]]+beta_u_new[validation_user[i]]
            prediction.append(predict_value)
            predict_value = 0
        s= 0
        for i in range(0,len(prediction)):
            s +=(prediction[i]-userrating_validation[i])**2
        MSE_new = s/len(prediction)
    MSE_list[lamda]=MSE_old
    print(lamda)

In [27]:
#QUESTION 8
beta_u_new = beta_u.copy()
beta_i_new = beta_i.copy()
lamda= 4.5
alpha = 4.23471
l = 0
m = 0
n = 0
MSE_old = 3
MSE_new= 2
validation_user = []
validation_item = []
count = 0
for d in validation_set:
        validation_user.append(d.split('-')[0])
        validation_item.append(d.split('-')[1])
while MSE_new < MSE_old:
    MSE_old = MSE_new
    for number in TR_dict:
        l += TR_dict[number]
    for user in beta_u:
        m = beta_u_new[user] + m
    for item in beta_i:
        n += beta_i_new[item]
    alpha_new = (l-m-n)/200000
    l = 0
    m = 0
    n = 0
    for user in beta_u_new:
        count_item = []
        l = sum(UR_dict[user])
        m = alpha_new*len(UR_dict[user])
        for item in UI_dict[user]:
            count_item.append(item)
        for x in count_item:
            n += beta_i_new[x]
        beta_u_new[user] = (l - m -n)/(lamda + len(UI_dict[user]))
        l = 0
        m = 0
        n = 0
    for k in beta_i_new:
        count_user = []
        l = sum(IR_dict[k])
        m = alpha_new*len(IR_dict[k])
        for user in IU_dict[k]:
            count_user.append(user)
        for x in count_user:
            n += beta_u_new[x]
        beta_i_new[k] = (l - m -n)/(lamda + len(IU_dict[k]))
        l = 0
        m = 0
        n = 0  
    prediction = []
    for i in range(0,len(validation_user)):
        if validation_user[i] in beta_u_new:
            user_judgement = True
        else:
            user_judgement = False
        if validation_item[i] in beta_i_new:
            item_judgement = True
        else:
            item_judgement = False
        if item_judgement == False and user_judgement == False :
            predict_value = alpha_new
        elif item_judgement == False and user_judgement == True:
            predict_value = alpha_new + beta_u_new[validation_user[i]]
        elif item_judgement == True and user_judgement ==False:
            predict_value = alpha_new + beta_i_new[validation_item[i]]
        else:
            predict_value = alpha_new + beta_i_new[validation_item[i]]+beta_u_new[validation_user[i]]
        prediction.append(predict_value)
        predict_value = 0
    s= 0
    for i in range(0,len(prediction)):
        s +=(prediction[i]-userrating_validation[i])**2
    MSE_new = s/len(prediction)
    count += 1
    print(MSE_new)

1.5427342694902821
1.555892053069471
